# Obtener datos ranking ELO

In [1]:
# https://www.worldfootball.net
# Librerías de siempre
import os
os.system("taskset -p 0xff %d" % os.getpid())

0

In [2]:
import pandas as pd
import datetime
from time import time
import requests
import urllib
from bs4 import BeautifulSoup, SoupStrainer
import re
import fnmatch
from joblib import Parallel, delayed
from joblib import parallel_backend

In [3]:
%%time
df_elo_test = pd.read_excel(os.path.join(os.path.pardir, 'Datos','Elo hasta 2006-12-08.xlsx'),
                            usecols = ['Date','Club','Elo'])

CPU times: user 6.36 s, sys: 37.6 ms, total: 6.4 s
Wall time: 6.93 s


In [4]:
df_elo_test

,Date,Club,Elo
0,1993-08-14,Man United,1799.935181
1,1993-08-14,Aston Villa,1703.455444
2,1993-08-14,Arsenal,1679.471680
3,1993-08-14,QPR,1676.099854
4,1993-08-14,Liverpool,1673.195679
5,1993-08-14,Blackburn,1665.707642
6,1993-08-14,Sheffield Weds,1664.222290
7,1993-08-14,Man City,1663.410156
8,1993-08-14,Wimbledon,1658.365601
9,1993-08-14,Sheffield United,1655.800293


In [6]:
def ObtenerRankingELO(fecha, country = 'ENG'):
    df_elo = pd.read_csv('http://api.clubelo.com/%s' % fecha, usecols = ['Club','Country','Elo'])
    df_elo = df_elo[df_elo['Country'] == country].reset_index(drop=True)
    df_elo['Date'] = fecha
    return df_elo[['Date','Club', 'Elo']]

In [59]:
df_cal = pd.read_excel(os.path.join(os.path.pardir,'Datos','Simulacion','Calibracion','Inglaterra.xlsx'))

In [8]:
%%time
fechas = df_cal['Date'].drop_duplicates().tolist()
fechas = [f for f in fechas if f not in df_elo_test['Date'].drop_duplicates().tolist()]
dfs_elo = []
errores = []
for fecha in fechas:
    print('Fecha actual: %s' % fecha, end ='\t\r')
    try:
        dfs_elo.append(ObtenerRankingELO(fecha, country = 'ENG'))
    except:
        errores.append(fecha)

CPU times: user 24.8 s, sys: 1.68 s, total: 26.5 s
Wall time: 1h 25min 30s


In [37]:
df_elo = pd.read_excel(os.path.join(os.path.pardir, 'Datos', 'ELO Inglaterra.xlsx'), usecols = ['Date','Club','Elo'])

In [55]:
prereplaces = {'Man ': 'Manchester ',
               'QPR': 'Queens Park Rangers',
               'Middlesboro': 'Middlesbrough',
               'Wolves': 'Wolverhampton Wanderers',
               'Sheffield Weds': 'Sheffield Wednesday'}
df_elo = df_elo.replace(prereplaces, regex = True)

In [40]:
df_elo.head()

,Date,Club,Elo
0,1993-08-14,Barnsley,1428.164795
1,1993-08-14,Swindon,1518.518311
2,1993-08-14,Leicester,1502.873047
3,1993-08-14,Tranmere,1485.208862
4,1993-08-14,Luton,1458.122803


In [56]:
equipos_elo = df_elo['Club'].drop_duplicates().tolist()
equipos_cal = df_cal['Local'].drop_duplicates().tolist()
dictreplaces = {}
for eq in equipos_elo:
    eq_cal = [i for i in equipos_cal if eq in i]
    if eq_cal:
        dictreplaces[eq] = eq_cal[0]

In [57]:
df_elo = df_elo.replace(dictreplaces, regex = True)

In [60]:
df_elo_l = df_elo.copy()
df_elo_l.columns = ['Date','Local','elo_local']
df_elo_v = df_elo.copy()
df_elo_v.columns = ['Date','Visita','elo_visita']

In [61]:
df_cal = df_cal.merge(df_elo_l, how = 'left', left_on = ['Date','Local'], right_on = ['Date','Local'])

In [62]:
df_cal = df_cal.merge(df_elo_v, how = 'left', left_on = ['Date','Visita'], right_on = ['Date','Visita'])

In [63]:
df_cal_check = df_cal[['Local','Visita','elo_local','elo_visita']]

In [64]:
df_cal_check[df_cal_check.isnull().any(axis=1)]

,Local,Visita,elo_local,elo_visita


In [65]:
df_cal.to_excel(os.path.join(os.path.pardir,'Datos','Simulacion','Calibracion','Inglaterra.xlsx'), index = False)

In [ ]:
def cleanhtml_tags(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
def obtener_links_tabla(tabla,html_root, tipo_tabla):
    table_rows = tabla.find_all('tr')
    urls = []
    banned = ['off','final','cup','abstieg', 'sued', 'nord', 'endrunde']
    for tr in table_rows:
        tds = tr.find_all('td')
        for td in tds:
            if  not any(x in str(td) for x in banned) and tipo_tabla in str(td):
                urls.append(td.find('a').get('href'))
    if tipo_tabla == 'competition':
        urls = [html_root + l.replace('competition','history') for l in urls]
    if tipo_tabla == 'schedule':
        urls = [html_root + l.replace('schedule','all_matches') for l in urls]
    return urls

## Obtener datos de estadios

In [ ]:
def info_estadios(url):
    html = urllib.request.urlopen(url).read()
    html_root = url.split('/venues')[0]
    soup = BeautifulSoup(html, features = "xml")
    col_names = ['Stadium', 'City', 'Country', 'Capacity','Venue']
    if 'Stadiums to be announced' not in str(soup):
        # Titulo de la liga
        liga = soup.findAll("title")
        liga = cleanhtml_tags(str(liga[0])).replace(' » Stadiums','').strip()
        options = soup.findAll("select")
        if 'Stadiums' in str(options):
            tables = soup.findAll("table") 
            table_rows = tables[0].find_all('tr')
            data = []
            for tr in table_rows:
                td = tr.find_all('td')
                ref = tr.find_all('a')
                row = [tr.text.strip() for tr in td if tr.text.strip()]
                if row:
                    row = [i.strip() for i in row if type(i) == str]
                    row = [i.replace("???","-") for i in row]
                    if ref:
                        row.append(html_root + str(ref[0].get('href')))
                    data.append(row)
            df_estadios = pd.DataFrame(data = data, columns = col_names)
            df_estadios['Torneo'] = liga.replace('Stadiums','').strip()
            df_estadios['Capacity'] = df_estadios['Capacity'].str.replace('.','')
            df_estadios['Capacity'] = df_estadios['Capacity'].replace('-',0, regex = False)
            df_estadios['Capacity'] = df_estadios['Capacity'].astype(int)           
    else:
        df_estadios = pd.DataFrame(columns = col_names)
    return df_estadios

In [ ]:
def obtener_ligas(url):
    html_root = url.split('/continents')[0]
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, features = "xml")
    tables = soup.findAll("table") 
    urls = obtener_links_tabla(tables[1],html_root, tipo_tabla = 'competition')
    return urls

In [ ]:
def obtener_torneos(url):
    html_root = url.split('/history')[0]
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, features = "xml")
    tables = soup.findAll("table") 
    urls = obtener_links_tabla(tables[0],html_root, tipo_tabla = 'competition')
    urls = [u.split('-spieltag')[0] for u in urls]
    urls = [u.replace('history','all_matches') for u in urls]
    return urls

In [ ]:
def obtener_metadatos(report_url):
    """
    Recibe una url del tipo:
    https://www.worldfootball.net/report/premier-league-2018-2019-manchester-united-leicester-city/
    y retorna la url de entrada, la url del estadio y la asistencia a un partido.
    """
    Venue, Attendance = "", ""
    try:        
        html = urllib.request.urlopen(report_url).read()
        html_root = report_url.split('/report')[0]
        strainer = SoupStrainer('table')
        soup = BeautifulSoup(html, 'lxml', parse_only=strainer)
        table = [t for t in soup.findAll("table") if 'venue' in str(t) and 'Attendance' in str(t)]
        table = table[0]
        trs = table.find_all('tr')
        ref = trs[0].find_all('a')
        if ref:
            Venue = html_root + str(ref[0].get('href'))
        tds = trs[1].find_all('td')
        Attendance = ''.join(c for c in tds[2].text if not c.isspace() and '.' not in c)
        if 'spectators' in Attendance:
            Attendance = 0
        try:
            Attendance = int(Attendance)
        except:
            pass
    except:
        pass
    return [report_url, Venue, Attendance]

## Obtener partidos de un torneo

In [ ]:
def obtener_tabla_temporada(url):
    html = urllib.request.urlopen(url).read()
    html_root = url.split('/all_matches')[0]
    soup = BeautifulSoup(html, features = "xml")
    tables = soup.findAll("table")
    liga = soup.findAll("title")
    info = cleanhtml_tags(str(liga[0])).split(' » ')
    Torneo = info[0]
    table_rows = [t for t in tables if 'teams' in str(t) and 'schedule' in str(t)][0].find_all('tr')
    return table_rows

In [ ]:
def obtener_partidos(url, metadata = True):
    """
    Función que recibe una url de worldfootball del tipo:
    https://www.worldfootball.net/all_matches/chi-primera-division-2018/
    Retorna un DataFrame que contiene las columnas:
    Fecha, Torneo, Round, Local, Visita, goles L, goles V, Report
    """
    html = urllib.request.urlopen(url).read()
    print('Url actual: %s                                                                    ' % url, end = '\t\r')
    html_root = url.split('/all_matches')[0]
    soup = BeautifulSoup(html, features = "xml")
    tables = soup.findAll("table")
    liga = soup.findAll("title")
    info = cleanhtml_tags(str(liga[0])).split(' » ')
    Torneo = info[0].replace('Schedule','').strip()
    table_rows = [t for t in tables if 'teams' in str(t) and 'schedule' in str(t)][0].find_all('tr')
    matches = []
    columns = ['Date','Torneo', 'Round', 'Local','Visita','goles L','goles V','Report']
    Fecha = ''
    Round = 0
    for tr in table_rows:
        tds = tr.find_all(['th','td'])
        if len(tds) < 2 and '<th' in str(tds[0]):
            Round = int(tds[0].find('a').text.replace('. Round',''))
        elif len(tds) > 2 and 'news' not in str(tds[0]):
            if 'matches' in str(tds[0]):
                dmy = ''.join(c for c in cleanhtml_tags(str(tds[0])) if not c.isspace() and '.' not in c).split('/')
                if len(dmy) == 3:
                    Fecha = dmy[2][:4] + '-' + dmy[1] + '-' + dmy[0]
            Local = tds[2].find('a').get('title')
            Visita = tds[4].find('a').get('title')
            outcomes = tds[5].text
            if '(' in outcomes:
                outcomes = outcomes.replace('(','').replace(')','').split(' ')
                final_outcome = outcomes[0].split(':')
            else:
                final_outcome = outcomes.split(':')
            if len(final_outcome) > 1:
                golesL = re.sub("[^0-9]", "", final_outcome[0])
                golesV = re.sub("[^0-9]", "", final_outcome[1])
            else:
                golesL = ""
                golesV = ""
            if 'report' in str(tds[5]):
                report_url = html_root + tds[5].find('a').get('href')
            else:
                report_url = ""
            if metadata:
                if golesL != "" and golesV != "":
                    matches.append([Fecha, Torneo, Round, Local, Visita, golesL, golesV, report_url])
            else:
                matches.append([Fecha, Torneo, Round, Local, Visita, golesL, golesV, report_url])
    df_matches = pd.DataFrame(data = matches, columns = columns)
    df_matches = df_matches.drop_duplicates()
    if metadata:
        df_matches['goles L'] = df_matches['goles L'].astype(int)
        df_matches['goles V'] = df_matches['goles V'].astype(int)
    else:
        df_matches = df_matches[['Date','Torneo','Round','Local','Visita']]
    return df_matches

In [ ]:
def exportar_data(leagues, outputdir):
    for league in leagues:
        filename = league + '.xlsx'
        url_liga = 'https://www.worldfootball.net/history/' + league + '/'
        print('--------------------------------------------------------------------------------------------------------')
        print('Generando: %s' % filename)
        ti = time()
        urls_torneos = obtener_torneos(url_liga)
        print('Número de torneos: %s' % len(urls_torneos))
        urls_estadios = [u.replace('all_matches','venues') for u in urls_torneos]
        dfs_matches = Parallel(n_jobs = 16)(delayed(obtener_partidos)(url) for url in urls_torneos)
        dfs_estadios = Parallel(n_jobs = 16)(delayed(info_estadios)(url) for url in urls_estadios)
        df_matches = pd.concat(dfs_matches, ignore_index = True)
        df_estadios = pd.concat(dfs_estadios, ignore_index = True, sort = True)
        urls_reports = df_matches['Report'].tolist()
        urls_reports = [u for u in urls_reports if u != ""]
        print('Número de reportes: %s' % len(urls_reports))
        metadata = Parallel(n_jobs = 60)(delayed(obtener_metadatos)(url) for url in urls_reports)
        print('Metadata obtenida')
        df_metadata = pd.DataFrame(data = metadata, columns = ['Report','Venue','Attendance'])
        df_matches = pd.merge(df_matches, df_metadata, how = 'left', on = ['Report'])
        df_matches = pd.merge(df_matches, df_estadios, how = 'left', on = ['Torneo','Venue'])
        df_matches = df_matches[[i for i in df_matches.columns if i not in ['Report','Venue']] + ['Report','Venue']]
        print('Guardando archivo')
        df_matches.to_excel(os.path.join(outputdir,filename), index = False)
        tf = time()
        deltat = str(datetime.timedelta(seconds=tf - ti))[:7]
        print('Ejecución terminada en %s' % deltat)

In [ ]:
outputdir = os.path.join(os.path.pardir,'Datos','Ligas')
leagues = ['arg-primera-division']
exportar_data(leagues, outputdir)

In [ ]:
# %%time
# df_fixture = obtener_partidos('https://www.worldfootball.net/all_matches/eng-premier-league-2018-2019/', metadata = False)

In [ ]:
# datadir = os.path.join(os.path.pardir, 'Datos','Fixtures')
# df_fixture.to_excel(os.path.join(datadir, 'eng-premier-league.xlsx'), sheet_name = 'Original', index = False)